# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [194]:
# imports
import panel as pn
pn.extension('plotly')
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
import os
from pathlib import Path
from dotenv import load_dotenv
from panel.interact import interact

In [195]:
# Read the Mapbox API key
load_dotenv()
map_box_api = os.getenv("mapbox")
px.set_mapbox_access_token(map_box_api)

# Import Data

In [196]:
# Import the necessary CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(file_path)

- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [197]:
# Define Panel Visualization Functions
#-----------------------------------Yearly Market Analysis--------------------------------------------------
def housing_units_per_year():
    """Housing Units Per Year."""
    
    units_per_year = sfo_data["housing_units"].groupby(sfo_data.index).mean()
    fig_housing_units = plt.figure()
    plot_housing_units = units_per_year.plot(
    kind='bar', 
    title='Housing Units in San Francisco from 2010 to 2016', 
    xlabel='Year', 
    ylabel='Units'
)

    housing_units_std = units_per_year.std()
    housing_units_min = min(units_per_year)
    housing_units_max = max(units_per_year)

    plot_housing_units.set_ylim(
        housing_units_min - housing_units_std, housing_units_max + housing_units_std
    )
    plt.close(fig_housing_units)
    return pn.pane.Matplotlib(fig_housing_units, tight=True)
#___________________________________________________________
def average_gross_rent():
    """Average Rent in San Francisco"""
    avg_rent = (
        sfo_data[["sale_price_sqr_foot", "gross_rent"]]
        .groupby(sfo_data.index)
        .mean()
    )

    fig_avg_gross_rent = plt.figure()
    plot_avg_gross_rent = avg_rent["gross_rent"].plot(color="green")
    plot_avg_gross_rent.set_xlabel("Year", fontsize=11)
    plot_avg_gross_rent.set_ylabel("Gross_Rent", fontsize=11)
    plot_avg_gross_rent.set_title(
        "Average Rent in San Francisco", fontsize=16, fontweight="bold"
    )


#     year_neighborhood = sfo_data.groupby(['year', 'neighborhood']).mean() #creates a subgroup in each year groupings


#     fig_avg_gross_rent = year_neighborhood.hvplot.line(
#     'year', 
#     'gross_rent', 
#     xlabel = 'Year', 
#     ylabel = 'Gross_Rent', 
#     title = 'Average Rent in San Francisco',
#     groupby = 'neighborhood',
# )


    plt.close(fig_avg_gross_rent)
    return pn.pane.Matplotlib(fig_avg_gross_rent, tight=True)
    # plt.close(fig_avg_gross_rent)
    # return pn.pane.Matplotlib(fig_avg_gross_rent, tight=True)

#_____________________________________________________________
def average_sales_price():
    """Average Sales Price Per Year."""
    
    df_avg_costs = (
        sfo_data[["sale_price_sqr_foot", "gross_rent"]]
        .groupby(sfo_data.index)
        .mean()
    )    
    
    fig_avg_sale_price = plt.figure()
    plot_avg_sale_price = df_avg_costs["sale_price_sqr_foot"].plot(color="red")
    plot_avg_sale_price.set_xlabel("Year", fontsize=11)
    plot_avg_sale_price.set_ylabel("Avg. Sale Price", fontsize=11)
    plot_avg_sale_price.set_title(
        "Average Sale Price per Square Foot", fontsize=16, fontweight="bold"
    )
    plt.close(fig_avg_sale_price)
    return pn.pane.Matplotlib(fig_avg_sale_price, tight=True)
#-----------------------------------------Yearly Market Analysis--------------------------------------------------

#-----------------------------------------Neighborhood Analysis---------------------------------------------------
def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    df_costs.rename(columns={"level_0": "year"}, inplace=True)
    

    sale_price_curve = df_costs.hvplot.line(
        "year",
        "sale_price_sqr_foot",
        xlabel="Year",
        ylabel="Avg. Sale Price per Square Foot",
        groupby="neighborhood",
    )


    return sale_price_curve
#_______________________________________________________________
def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""

    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False
    ).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()

    expensive_neighborhoods = df_expensive_neighborhoods.hvplot.bar(
        "neighborhood",
        "sale_price_sqr_foot",
        title="Top 10 Expensive Neighborhoods in San Francisco",
        xlabel="Neighborhood",
        ylabel="Avg Sale Price per SqFt",
        height=550,
        rot=45,
    )
    return expensive_neighborhoods
#______________________________________________________________________
def most_expensive_neighborhoods_rent_sales():
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    df_costs.rename(columns={"level_0": "year"}, inplace=True)
    
    rent_sales_plot = df_costs.hvplot.bar(
    "year",
    ["gross_rent","sale_price_sqr_foot"],
    title="Top 10 Expensive Neighborhoods in San Francisco",
    xlabel="Neighborhood",
    ylabel="Num Housing Units",
    groupby='neighborhood',
    height=550,
    rot=45)
    return rent_sales_plot
    
#-----------------------------------------Neighborhood Analysis--------------------------------------------------

#-----------------------------------------Parallel Plots Analysis------------------------------------------------
def parallel_coordinates():
    """Parallel Coordinates Plot."""

    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False
    ).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    
    expensive_coordinates_plot = px.parallel_coordinates(
        df_expensive_neighborhoods,
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        title="Parallel Coordinates of Most Expensive San Francisco Neighborhoods",
        height = 550, 
        width = 800
    )
    return expensive_coordinates_plot

#_________________________________________________________________________
def parallel_categories():
    """Parallel Categories Plot."""
    
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False
    ).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    
    expensive_categories_plot = px.parallel_categories(
        df_expensive_neighborhoods,
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        title="Parallel Categories Analysis of Most Expensive San Francisco Neighborhoods",
        height = 550, 
        width = 800
    )
    return expensive_categories_plot
#-----------------------------------------Parallel Plots Analysis--------------------------------------------------

#-------------------------------------------------WELCOME-----------------------------------------------------
def neighborhood_map():
    """Neighborhood Map"""
    px.set_mapbox_access_token(map_box_api)
    
    df_all_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_all_neighborhoods.reset_index(inplace=True)
    df_all_neighborhoods.rename(columns={"neighborhood": "Neighborhood"}, inplace=True)

    df_all_neighborhoods = pd.merge(
        df_neighborhood_locations, df_all_neighborhoods, on="Neighborhood"
    )

    neighborhood_map = px.scatter_mapbox(
        df_all_neighborhoods,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="gross_rent",
        color_continuous_scale=px.colors.cyclical.IceFire,
        size_max=22,
        zoom=11,
        hover_name="Neighborhood",
        title="Average Sale Price Per Square Foot and Gross Rent in San Francisco",
    )

    plotly_panel = pn.pane.Plotly(neighborhood_map)
    plotly_panel._updates = True
    return plotly_panel
#-----------------------------------------Sunburst--------------------------------------------------
def sunburst():
    df_costs = sfo_data.groupby([sfo_data.index, "neighborhood"]).mean()
    df_costs.reset_index(inplace=True)
    df_costs.rename(columns={"level_0": "year"}, inplace=True)
    df_expensive_neighborhoods = sfo_data.groupby(by="neighborhood").mean()
    df_expensive_neighborhoods = df_expensive_neighborhoods.sort_values(
        by="sale_price_sqr_foot", ascending=False
    ).head(10)
    df_expensive_neighborhoods = df_expensive_neighborhoods.reset_index()
    df_expensive_neighborhoods_per_year = df_costs[df_costs["neighborhood"].isin(df_expensive_neighborhoods["neighborhood"])]
    fig_sunburst = px.sunburst(
        df_expensive_neighborhoods_per_year,
        path=["year", "neighborhood"],
        values='sale_price_sqr_foot',
        color='gross_rent', 
        color_continuous_scale='Blues',
        title="Costs Analysis of Most Expensive neighborhoods in San Francisco per Year",
        height=550)
    plotly_panel_sunburst = pn.pane.Plotly(fig_sunburst)
    plotly_panel_sunburst._updates = True
    return plotly_panel_sunburst

## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [198]:
# Create a Title for the Dashboard
title = pn.pane.Markdown(
    """
# 2010 to 2016 San Francisco Real Estate Analysis
""",
    width=500,
)

welcome = pn.pane.Markdown(
    """
    Welcome to the historical housing unit prices of San Fransisco Dashboard.
    Please use the tabs above to navigate different details of the market. 
    The data utilized for this dashboards ranges from 2010 to 2016 historical records. 
"""
)

# Create a tab layout for the dashboard
tabs = pn.Tabs(
    ("Welcome", pn.Column(welcome, neighborhood_map())),
    ("Yearly Market Analysis", pn.Column(housing_units_per_year(), average_gross_rent(), average_sales_price())),
    #("Yearly Market Analysis", pn.Row(housing_units_per_year(), average_gross_rent(), average_sales_price())),
    ("Neighborhood Analysis", pn.Column(average_price_by_neighborhood(), top_most_expensive_neighborhoods(), most_expensive_neighborhoods_rent_sales())),
    ("Parallel Plots Analysis", pn.Column(parallel_coordinates(), parallel_categories(),width=500)),
    ("Sunburst Plot Analysis", pn.Column(sunburst())),
    )

dashboard = pn.Column(pn.Row(title), tabs, width=500)

## Serve the Panel Dashboard

In [199]:
# Serve the# dashboard
dashboard.servable()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0db2e280-d27a-449a-846e-b45865426be3': {'defs': …

Column(width=500)
    [0] Row
        [0] Markdown(str, width=500)
    [1] Tabs
        [0] Column
            [0] Markdown(str)
            [1] Plotly(Figure)
        [1] Column
            [0] Matplotlib(Figure, tight=True)
            [1] Matplotlib(Figure, tight=True)
            [2] Matplotlib(Figure, tight=True)
        [2] Column
            [0] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
            [1] HoloViews(Bars)
            [2] Row
                [0] HoloViews(DynamicMap)
                [1] Column
                    [0] WidgetBox
                        [0] Select(margin=(20, 20, 20, 20), name='neighborhood', options=['Alamo Square', ...], value='Alamo Square', width=250)
                    [1] VSpacer()
        [3] Column(width=500)
            [0] Plotly(Figure)
            [1] Plotly(Figure)
        [4] Column
            [0] Plotly(Figure)

# Debugging

Note: Some of the Plotly express plots may not render in the notebook through the panel functions.

However, you can test each plot by uncommenting the following code

In [200]:
# housing_units_per_year()

In [201]:
# average_gross_rent()

In [202]:
# average_sales_price()

In [203]:
# average_price_by_neighborhood()

In [204]:
# top_most_expensive_neighborhoods()

In [205]:
# most_expensive_neighborhoods_rent_sales()

In [206]:
# neighborhood_map().show()

In [207]:
# parallel_categories()

In [208]:
# parallel_coordinates()

In [209]:
# sunburst()